In [25]:
(** 1. 정수 덧셈만 가능한 계산기 *)
(* 더할 정수들을 리스트로 받는다. *)

module Calc_v1 = struct 
  let calc : int list -> int 
  = fun num_list -> 
  let add x y = x + y in
  List.fold_left add 0 num_list

end

module Calc_v1 : sig val calc : int list -> int end


In [5]:

  
  (* num_list = [i1; i2; ... ; in] 이라고 할 때, (단 i1 ... in 은 정수) *)
  (* List.fold_left add 0 [i1; i2; ... ; in] = (add (add (add (add 0 i1) i2) i3) ... in) *)
  
  (* 혹은 아래와 같이 재귀함수를 이용하여 정의 가능 *)
  let rec calc 
  = fun num_list -> match num_list with
  |[] -> 0
  | hd::tl -> hd + (calc tl)


val calc : int list -> int = <fun>


In [6]:
["1"; "+"; "3"; "-"; "2" ];;

- : string list = ["1"; "+"; "3"; "-"; "2"]


In [24]:
module Calc_v2 = struct
  let rec calc : string list -> int
  = fun str_list -> 
  match str_list with
  |[] -> 0
  |hd::hd2::tl -> 
  let v = int_of_string hd2 in
  if (String.compare hd2 "+") = 0 then
    v + (calc tl)
  else 
    v - (calc tl)
| hd :: tl -> int_of_string hd 
end

module Calc_v2 : sig val calc : String.t list -> int end


In [11]:
["1"; "+"; "a"];;

- : string list = ["1"; "+"; "a"]


사용자 정의 타입

In [13]:
type exp = INT of int | ADD | SUB;;

[INT(1);ADD;INT(3);SUB;INT(2)];;


type exp = INT of int | ADD | SUB


- : exp list = [INT 1; ADD; INT 3; SUB; INT 2]


In [19]:
let get_num : exp -> int
= fun e ->
match e with 
| INT n -> n
| _ -> failwith "invaid"

let rec calc : exp list -> int
  = fun lst -> 
  match lst with
  |[] -> 0
  |hd1::hd2::tl ->
  (
  match hd2 with 
    | ADD -> (get_num hd1) + (calc tl)
    | SUB -> (get_num hd1) - (calc tl)
    | INT n -> failwith "invalid"
    )
    | hd1 :: tl -> (get_num hd1);;
    
calc [INT(1);ADD;INT(3);SUB;INT(2)];;

val get_num : exp -> int = <fun>


val calc : exp list -> int = <fun>


- : int = 2


In [20]:
(* bug *)
calc [INT(1);ADD;INT(3);SUB]

- : int = 4


3 곱셈추가

In [23]:
module Calc_v3 = struct 
  type exp = 
  |INT of int | ADD of exp * exp | SUB of exp * exp | MUL of exp * exp
  
  let rec calc: exp -> int 
  = fun e -> 
  match e with 
  | INT n -> n
  | ADD (e1, e2) -> (calc e1) + (calc e2)
  | SUB (e1, e2) -> (calc e1) - (calc e2)
  | MUL (e1, e2) -> (calc e1) * (calc e2)
  
let _ =
  let e = MUL(ADD(INT(1), INT(3)), INT(2)) in
  let v = calc e in
  Printf.printf "%d\n" v
  
end
  

module Calc_v3 :
  sig
    type exp =
        INT of int
      | ADD of exp * exp
      | SUB of exp * exp
      | MUL of exp * exp
    val calc : exp -> int
  end


4. 변수추가

In [34]:
module Calc_v4 = struct 
type exp = 
|INT of int | ADD of exp * exp | SUB of exp * exp | MUL of exp * exp

let rec calc : exp -> int -> int  
  = fun e x ->
  match e with
  | x -> x
  | INT n -> n
  | ADD (e1, e2) -> (calc e1 x) + (calc e2 x)
  | SUB (e1, e2) -> (calc e1 x) - (calc e2 x)
  | MUL (e1, e2) -> (calc e1 x) * (calc e2 x)
  
  let _ =
  let e = MUL(ADD(X, INT(3)), INT(2)) in
  let v = calc e 3 in
  Printf.printf "%d\n" v
end

error: compile_error

5 변수 여러개 추가

In [38]:
(** 5. 변수 여러개 추가 *)
(* 변수 여러개로 표현되는 식과 각 변수들의 값을 받아서 계산 결과를 내놓는 계산기 *)
(* 예: calc (x + y + 3) {x -> 1, y -> 2} = 6 *)
module Calc_v5 = struct 
  type exp = VAR of string |INT of int | ADD of exp * exp | SUB of exp * exp | MUL of exp * exp
  
	(* 변수 ID 에서 정수 값으로의 맵핑이 필요. *)
	type environment = (string * int) list 
	(* 예) [("x", 1); ("y", 2)]: x와 y에 각각 1과 2 할당.  *)

	(* exp 와 env 를 인자로 받음. *)
  let rec calc: exp -> environment -> int 
  = fun e env -> 
  match e with 
  | VAR id -> List.assoc id env
  | INT n -> n
  | ADD (e1, e2) -> (calc e1 env) + (calc e2 env)
  | SUB (e1, e2) -> (calc e1 env) - (calc e2 env)
  | MUL (e1, e2) -> (calc e1 env) * (calc e2 env)
  
	(* test *)
  let _ =
  (* *)
  	let e = MUL(ADD((VAR("x")), INT(3)), VAR("y")) in
  	let v = calc e [("x", 1); ("y", 2)] in
  	Printf.printf "%d\n" v
end

module Calc_v5 :
  sig
    type exp =
        VAR of string
      | INT of int
      | ADD of exp * exp
      | SUB of exp * exp
      | MUL of exp * exp
    type environment = (string * int) list
    val calc : exp -> environment -> int
  end


In [35]:
List.assoc "x" [("x", 1); ("y", 2)];;

- : int = 1


In [36]:
List.assoc "x" [("x", 0);("x", 1); ("y", 2)];;

- : int = 0


6 let 표현식 추가. 
변수마다 특정범위의 특정값으로 바인딩

In [40]:
(** 6. let 표현식 추가.  *)
(* 예: (let x = 2 in x + 1) * (let y = 3 in y + 1) = 12 *)
module Calc_v6 = struct 
  type exp = 
  | LET of string * exp * exp
  | VAR of string |INT of int | ADD of exp * exp | SUB of exp * exp | MUL of exp * exp
  
	(* 변수 ID 에서 정수 값으로의 맵핑이 필요. *)
	type environment = (string * int) list
	(* 예) [("x", 1); ("y", 2)]: x와 y에 각각 1과 2 할당.  *)

	(* exp 와 env 를 인자로 받음. *)
  let rec calc: exp -> environment -> int 
  = fun e env -> 
  match e with
  | LET (vid, e1, e2)->
  let v = calc e1 env in
  let env' = (vid, v) :: env in
  (calc e2 env')
  | VAR id -> List.assoc id env
  | INT n -> n
  | ADD (e1, e2) -> (calc e1 env) + (calc e2 env)
  | SUB (e1, e2) -> (calc e1 env) - (calc e2 env)
  | MUL (e1, e2) -> (calc e1 env) * (calc e2 env)
  
	(* test *)
  let _ =
		(* let x = 3 in let y = 4 in x + y *)
  	let e = LET("x", INT(3), LET("y", INT(4), ADD(VAR("x"), VAR("y"))))  in
  	let v = calc e [] in
  	Printf.printf "%d\n" v;
    (* let x = 3 in let x = 4 in x + x *)
  	let e = LET("x", INT(3), LET("x", INT(4), ADD(VAR("x"), VAR("x"))))  in
  	let v = calc e [] in
  	Printf.printf "%d\n" v;
end 

module Calc_v6 :
  sig
    type exp =
        LET of string * exp * exp
      | VAR of string
      | INT of int
      | ADD of exp * exp
      | SUB of exp * exp
      | MUL of exp * exp
    type environment = (string * int) list
    val calc : exp -> environment -> int
  end


7 

In [45]:
(** 7. 조건문 IF 추가. *)
(* 예: let x = 3 in if (x <= 4) then x + 1 else x + 2 *)
module Calc_v7 = struct 
  type exp = 
  | IF of exp * exp * exp
  | LET of string * exp * exp
  | VAR of string |INT of int | ADD of exp * exp | SUB of exp * exp | MUL of exp * exp
  | BOOL of bool 
  | EQUAL of exp * exp (* *)
  | LESS of exp * exp
  | NOT of exp
  
  
	(* 변수 ID 에서 정수 값으로의 맵핑이 필요. *)
	type environment = (string * int) list
	(* 예) [("x", 1); ("y", 2)]: x와 y에 각각 1과 2 할당.  *)

let rec eval_cond : exp -> environment -> bool
= fun e env ->
match e with
| BOOL b -> b
| NOT e' -> not (eval_cond e' env)
| EQUAL (e1,e2) ->
  let v1 - calc e1 env in
  let v2 - calc e2 env in
  v1 = v2
| LESS (e1,e2) ->
  let v1 - calc e1 env in
  let v2 - calc e2 env in
  v1 = v2

	(* exp 와 env 를 인자로 받음. *)
  and calc: exp -> environment -> int 
  = fun e env -> 
  match e with 
  | IF(cond, then_branch, else_branch) -> 
  let b = eval_cond cond in
  if (b) then (calc then_branch env) else (calc else_branch env)
  | LET (vid, e1, e2)->
  let v = calc e1 env in
  let env' = (vid, v) :: env in
  (calc e2 env')
  | VAR id -> List.assoc id env
  | INT n -> n
  | ADD (e1, e2) -> (calc e1 env) + (calc e2 env)
  | SUB (e1, e2) -> (calc e1 env) - (calc e2 env)
  | MUL (e1, e2) -> (calc e1 env) * (calc e2 env)
  
	
	(* test *)
  let _ =
		(* let x = 3 in let y = 4 in if (x = y) then 1 else 2 *)
  	let e = LET("x", INT(3), LET("y", INT(4), IF(EQUAL(VAR("x"), VAR("y")), INT(1), INT(2))))  in
  	let v = calc e [] in
  	Printf.printf "%d\n" v;
end 		

error: compile_error

In [47]:
(** 8. 매개변수 1개인 함수 추가. *)
module Calc_v8 = struct 
  type exp = 
  | LETF of string * string * exp * exp 
  | CALL of string * exp 
  | IF of exp * exp * exp
  | LET of string * exp * exp
  | VAR of string |INT of int | ADD of exp * exp | SUB of exp * exp | MUL of exp * exp
  | BOOL of bool 
  | EQUAL of exp * exp (* *)
  | LESS of exp * exp
  | NOT of exp
  
	(* 변수 ID 에서 값으로의 맵핑이 필요. *)
	type value_or_func = Num of int | Func of string * exp   
	type environment = (string * value_or_func) list
	(* 예) [("x", Num 1); ("f", Func ("a1", ADD(VAR("a1"), INT(1))))]: x 에 1, f 에 매개변수 a1 함수 몸체 a1 + 1 인 함수 할당 *)

	(* exp 와 env 를 인자로 받음. *)
  let rec calc: exp -> environment -> int 
  = fun e env -> 
  match e with
  |LETF (fic, pid, body, e) ->
  let env' = (fic, fun(pid, body))::env in
  (calc e env')
  |CALL (fid, arg)->
  match (Lit.assoc fid env) with
| Num _ -> failwith "safdasfd"
|Func(pid, body) ->
calc (LET (pid, arg, body)) env

)
| IF(cond, then_branch, else_branch) -> 
  let b = eval_cond cond in
  if (b) then (calc then_branch env) else (calc else_branch env)
  | LET (vid, e1, e2)->
  let v = calc e1 env in
  let env' = (vid, Num(v)) :: env in
  (calc e2 env')
  | VAR id -> 
  (match (  List.assoc id env ) with
  | Num n -> n
  | _ -> failwith "sfsafd"
  )
  | INT n -> n
  | ADD (e1, e2) -> (calc e1 env) + (calc e2 env)
  | SUB (e1, e2) -> (calc e1 env) - (calc e2 env)
  | MUL (e1, e2) -> (calc e1 env) * (calc e2 env)
  | _ -> failwith "dsfasfd"
  
  and eval_cond : exp -> environment -> bool
= fun e env ->
match e with
| BOOL b -> b
| NOT e' -> not (eval_cond e' env)
| EQUAL (e1,e2) ->
  let v1 - calc e1 env in
  let v2 - calc e2 env in
  v1 = v2
| LESS (e1,e2) ->
  let v1 - calc e1 env in
  let v2 - calc e2 env in
  v1 = v2
  
	(* test *)
  let _ =
		(* let f(a) = if (a <= 10) then f(a + 1) else a in f(2) *)
		let body = IF(LESS(VAR("a"), INT(10)), CALL("f", ADD(VAR("a"), INT(1))), VAR("a")) in
  	let e = LETF ("f", "a", body, CALL("f", INT(2)))  in
  	let v = calc e [] in
  	Printf.printf "%d\n" v
		(* 참고: 현재 구현은 dynamic scope 방식을 따름: 함수 몸체가 함수가 호출 될 때의 환경을 참조하여 실행됨. *)
		(* 이에 반해 static scope 방식은 함수 몸체가 함수가 정의될 때의 환경을 참조하여 실행됨.  *)
		(* 예) *)
		(*   let x = 2 in  *)
		(*   let f(a) = x + a in *)
		(*   let x = 3 in f(1) *)
		(* static(정의될때?) scope 은 3(ocaml), dynamic scop때(호출될때?) 은 4를 계산 *)
end 

error: compile_error

In [ ]:
		(*   let x = 2 in  *)
		(*   let f(a) = x + a in *)
		(*   let x = 3 in f(1) *)
		(* static scope 은 3, dynamic scope 은 4를 계산 *)

In [ ]:
(** 9. 파서를 이용하여 파일을 읽어들여 실행  *)
(* interpret_from_file.ml , lexer.mll, parser.mly 참조. *)
(* interpret_from_file.ml 컴파일 후 (e.g., ocamlbuild interpret_from_file.byte), 명령문 파일과 함께 실행. *)
(* 실행 예) ./interpret_from_file.byte test1.calc *)
(* 참고: ocamllex, ocamlyacc 은 lex, yacc (혹은 bison)과 거의 흡사한 lexer, parser generator. *)
(* (https://caml.inria.fr/pub/docs/manual-ocaml-4.06/lexyacc.html) *)


(** (실습 1.) Calc_v8 을 확장하여 나눗셈 연산과, 외부 입력으로부터 정수를 받는 READ 구문을 추가하세요 (read_int() 함수 사용) *)
(** (실습 2.) Calc_v8 을 확장하여 함수가 임의의 갯수의 인자를 받을 수 있도록 하세요. *)


In [1]:
read_int();;

error: runtime_error

In [ ]:
(** (실습 3.) Calc_v8 을 확장하여 SWITCH 구문을 추가하세요. 
    SWITCH (e, [(e1, e1'), (e2, e2'), ... (en, en')])
		의미: e의 결과가 e1 이면 e1' 실행, e2이면 e2'실행 ...  
	*)
    